In [ ]:
import numpy as np
from scipy.optimize import minimize, NonlinearConstraint

def hng_loglik(params, r):
    """
    Compute the negative log-likelihood of the Heston–Nandi GARCH(1,1) model.

    params: array-like, [omega, alpha, beta, gamma, lambd]
      omega > 0
      alpha >= 0
      beta >= 0
      gamma (leverage parameter)
      lambd (market price of risk)
    r: array-like of log‐returns
    """
    omega, alpha, beta, gamma, lambd = params

    T = len(r)
    # initialize conditional variance
    h = np.zeros(T)
    # start at unconditional variance
    h[0] = (omega_true + alpha_true) / (1 - beta_true - alpha_true*(gamma_true**2))
    h[0] = np.var(r)  # ensure positive variance

    # innovations
    z = np.zeros(T)

    for t in range(1, T):

        # standardized residual (under P-measure)
        z[t-1] = (r[t-1] - lambd * h[t-1]) / np.sqrt(h[t-1])
        # variance recursion
        h[t] = (
            omega
            + beta * h[t-1]
            + alpha * (z[t-1] - gamma * np.sqrt(h[t-1]))**2
        )

    # for last residual
    z[-1] = (r[-1] - lambd * h[-1]) / np.sqrt(h[-1])

    # log-likelihood
    ll = -0.5 * (np.log(2 * np.pi) + np.log(h) + z**2).sum()
    # we minimize negative LL
    return -ll


def fit_heston_nandi(r, start_vals=None):
    """
    Fit Heston–Nandi GARCH(1,1) to return series r.
    Returns OptimizeResult with .x = [omega, alpha, beta, gamma, lambd]
    """
    # initial guesses if none provided
    if start_vals is None:
        var_r = np.var(r)
        start_vals = np.array([0.000001, 0.05, 0.90, 0.0, 0.0])

    # bounds: omega>0, alpha>=0, beta>=0, gamma free, lambd free
    bnds = [
        (1e-12, None),   # omega
        (0.0, 1.0),      # alpha
        (0.0, 1.0),      # beta
        (None, None),    # gamma
        (None, None),    # lambd
    ]

    result = minimize(
        fun=hng_loglik,
        x0=start_vals,
        args=(r,),
        bounds=bnds,
        method='L-BFGS-B',
        options={'disp': False, 'maxiter': 500}
    )
    return result


if __name__ == "__main__":

    # example usage with simulated data
    np.random.seed(1)

    # number of observations
    T = 2500

    # true parameters
    omega_true = 1e-6
    alpha_true = 0.01
    beta_true  = 0.9
    gamma_true = 1
    lambd_true = 0.5

    # simulate HNG process
    h = np.zeros(T)
    r = np.zeros(T)
    h[0] = (omega_true + alpha_true) / (1 - beta_true - alpha_true*(gamma_true**2))
    print("true initial variance", h[0])

    for t in range(1, T):
        z = np.random.randn()
        r[t-1] = lambd_true * h[t-1] + np.sqrt(h[t-1]) * z
        h[t] = (
            + omega_true
            + beta_true * h[t-1]
            + alpha_true * (z - gamma_true * np.sqrt(h[t-1]))**2
        )
    z = np.random.randn()
    r[-1] = lambd_true * h[-1] + np.sqrt(h[-1]) * z

    # fit model
    res = fit_heston_nandi(r)
    print("Estimated parameters:")
    print("omega =", res.x[0])
    print("alpha =", res.x[1])
    print("beta  =", res.x[2])
    print("gamma =", res.x[3])
    print("lambda=", res.x[4])


0.11112222222222223
Estimated parameters:
omega = 0.0015908018386082958
alpha = 0.01075484984815177
beta  = 0.8766044092994955
gamma = 1.0670402265630847
lambda= 0.5632741009301748


In [ ]:
import numpy as np
from scipy.optimize import minimize, NonlinearConstraint

def hng_loglik(params, r):
    """
    Compute the negative log-likelihood of the Heston–Nandi GARCH(1,1) model.

    params: array-like, [omega, alpha, beta, gamma, lambd]
      omega > 0
      alpha >= 0
      beta >= 0
      gamma (leverage parameter)
      lambd (market price of risk)
    r: array-like of log‐returns
    """
    omega, alpha, beta, gamma, lambd = params

    T = len(r)
    # initialize conditional variance
    h = np.zeros(T)
    # start at unconditional variance
    h[0] = omega / (1 - alpha * (1 + gamma**2) - beta)

    # innovations
    z = np.zeros(T)

    for t in range(1, T):

        # standardized residual (under P-measure)
        z[t-1] = (r[t-1] - lambd * h[t-1]) / np.sqrt(h[t-1])
        # variance recursion
        h[t] = (
            omega
            + beta * h[t-1]
            + alpha * (z[t-1] - gamma * np.sqrt(h[t-1]))**2
        )

    # for last residual
    z[-1] = (r[-1] - lambd * h[-1]) / np.sqrt(h[-1])

    # log-likelihood
    ll = -0.5 * (np.log(2 * np.pi) + np.log(h) + z**2).sum()
    # we minimize negative LL
    return -ll


def fit_heston_nandi(r, start_vals=None):
    """
    Fit Heston–Nandi GARCH(1,1) to return series r.
    Returns OptimizeResult with .x = [omega, alpha, beta, gamma, lambd]
    """
    # initial guesses if none provided
    if start_vals is None:
        start_vals = np.array([0.000001, 0.05, 0.90, 0.0, 0.0])

    # bounds: omega>0, alpha>=0, beta>=0, gamma free, lambd free
    bnds = [
        (1e-12, None),   # omega
        (0.0, 1.0),      # alpha
        (0.0, 1.0),      # beta
        (None, None),    # gamma
        (None, None),    # lambd
    ]

    def constraint_ineq(x):
        (1 - alpha_true * (1 + gamma_true**2) - beta_true)
        return (1 - x[1] * (1 + x[3]**2) - x[2])
    
    a_constraints = (
    {'type': 'ineq', 'fun': constraint_ineq}
)

    result = minimize(
        fun=hng_loglik,
        x0=start_vals,
        args=(r,),
        bounds=bnds,
        method='SLSQP',
        constraints=a_constraints,
        options={'disp': False, 'maxiter': 500}
    )

    return result


if __name__ == "__main__":
    # Example usage with simulated data
    np.random.seed(2)
    T = 25000
    # true parameters
    omega_true = 1e-6
    alpha_true = 0.07
    beta_true  = 0.92
    gamma_true = -0.0
    lambd_true = 0.5

    print("initial true variance:", (1 - alpha_true * (1 + gamma_true**2) - beta_true))

    # simulate HNG process
    h = np.zeros(T)
    r = np.zeros(T)
    h[0] = omega_true / (1 - alpha_true * (1 + gamma_true**2) - beta_true)
    print(h)
    for t in range(1, T):
        z = np.random.randn()
        r[t-1] = lambd_true * h[t-1] + np.sqrt(h[t-1]) * z
        h[t] = (
            omega_true
            + beta_true * h[t-1]
            + alpha_true * (z - gamma_true * np.sqrt(h[t-1]))**2
        )
    r[-1] = lambd_true * h[-1] + np.sqrt(h[-1]) * np.random.randn()

    # fit model
    res = fit_heston_nandi(r)
    print("Estimated parameters:")
    print("omega =", res.x[0])
    print("alpha =", res.x[1])
    print("beta  =", res.x[2])
    print("gamma =", res.x[3])
    print("lambda=", res.x[4])


initial true variance: 0.009999999999999898
[0.0001 0.     0.     ... 0.     0.     0.    ]


/var/folders/5g/b4bpmvtn7tjcmrg4l2k_p_9r0000gn/T/ipykernel_10185/485578670.py:30: RuntimeWarning: invalid value encountered in sqrt
  z[t-1] = (r[t-1] - lambd * h[t-1]) / np.sqrt(h[t-1])
/var/folders/5g/b4bpmvtn7tjcmrg4l2k_p_9r0000gn/T/ipykernel_10185/485578670.py:35: RuntimeWarning: invalid value encountered in sqrt
  + alpha * (z[t-1] - gamma * np.sqrt(h[t-1]))**2
/var/folders/5g/b4bpmvtn7tjcmrg4l2k_p_9r0000gn/T/ipykernel_10185/485578670.py:42: RuntimeWarning: invalid value encountered in log
  ll = -0.5 * (np.log(2 * np.pi) + np.log(h) + z**2).sum()


Estimated parameters:
omega = 2.607627207357138
alpha = 0.0
beta  = 0.0
gamma = -4.395713264865121
lambda= 0.1679391020187765


In [9]:
(1 - alpha_true * (1 + gamma_true**2) - beta_true)

-0.06000000000000005